In [2]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from fpdf import FPDF
import numpy as np
import nipype
import nipype.interfaces.fsl as fsl

In [3]:
#go to data
mri_dir = '/gpfs/milgram/pi/gee_dylan/candlab/data/mri/bids_recon/shapes'
TW1_image_location = '/home/gg558/TW1_images/'
TW2_image_location = '/home/gg558/TW2_images/'
mni_dir = '/gpfs/milgram/pi/gee_dylan/candlab/atlases/MNI152_T1_2mm_brain.nii.gz'

In [ ]:
#create two PDFs
T1w_pdf = FPDF()
T1w_pdf.add_page()

T2w_pdf = FPDF()
T2w_pdf.add_page()

#go through every subject
for file in os.listdir(mri_dir):
    #for every subject folder, go into the session folder
    if file.startswith('sub'):
        subject_ID = os.fsdecode(file)
        subject_dir = mri_dir + '/' + subject_ID + '/ses-shapesV1/anat/'
    
        #go through every file within the session folder
        for subfile in os.listdir(subject_dir):
            data_type = os.fsdecode(subfile)
            data_dir = subject_dir + '/' + subfile
        
            #get to the FA data
            if data_dir.endswith('T1w.nii.gz') or data_dir.endswith('T2w.nii.gz'):
                #reorient anats to MNI space before slicing
                aw = fsl.ApplyWarp()
                aw.inputs.in_file = data_dir
                aw.inputs.ref_file = mni_dir
                aw.inputs.out_file = data_dir
                reoriented_image = aw.run() 
                
                #do the slicer 
                slicer = fsl.Slicer()
                slicer.inputs.in_file = data_dir
                slicer.inputs.middle_slices = True
                slicer.inputs.out_file = subject_ID + '.png'
                res = slicer.run() 
                slicer.inputs.scaling = 100

                #save as a PNG
                img = mpimg.imread(subject_ID + '.png')
                imgplot = plt.imshow(img)
                fig1 = plt.gcf()
                plt.axis('off')
                plt.title("Subject" + ' ' + subject_ID.split('-',1)[1])
                
                #save PNG files to correct folder
                if data_dir.endswith('T1w.nii.gz'):
                    os.chdir(TW1_image_location)
                    plt.savefig(TW1_image_location + subject_ID + '.png', dpi = 1200)
                if data_dir.endswith('T2w.nii.gz'):
                    os.chdir(TW2_image_location)
                    plt.savefig(TW2_image_location + subject_ID + '.png', dpi = 1200)
                
                plt.close()

                #add PNG to correct PDF file based on T1 or T2
                if data_dir.endswith('T1w.nii.gz'):
                    T1w_pdf.image(TW1_image_location + subject_ID + '.png')
                if data_dir.endswith('T2w.nii.gz'):
                    T2w_pdf.image(TW2_image_location + subject_ID + '.png')
            
            #continue on with the next iteration
            else:
                continue 

#save both PDFs of data
T1w_pdf.output('/home/gg558/Anat_T1w_data.pdf')
T2w_pdf.output('/home/gg558/Anat_T2w_data.pdf')